In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (16, 3)
plt.rcParams["axes.grid"] = True

df = pd.read_csv("../csv/global_returns.csv", delimiter=';')
ds = pd.read_csv("../csv/session.csv", delimiter=';')

In [ ]:
print("Available data")
print("==============")
print(f"{df.shape[0]:>8} function returns")
print(f"{ds.shape[0]:>8} session datapoints")

In [ ]:
print(f"ALS start time: {df.iloc[0,0][:-4]}")
print(f"ALS stop time:  {df.iloc[-1,0][:-4]}")

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
mem=ds[ds.type == 'memory'].copy()
mem['value'] = mem['value'].apply(lambda m: m/1024/1024)

mem.plot(title="Available memory",
    x='timestamp',
    y="value",
    xlabel='',
    ylabel="MB",
    legend=False,
    figsize=(16, 4))
plt.show()

In [ ]:
df["module"].value_counts().plot(kind="bar", title="Function calls by module")
plt.show()

In [ ]:
df['thread'].value_counts().plot(kind="pie", title="Function calls by thread", figsize=(6, 6))
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.scatterplot(data=df,
       x='timestamp',
       y='module', hue='thread')
ax.set_title("Activity flow by module")
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.scatterplot(data=df,
       x='timestamp',
       y='thread', hue='module')
ax.set_title("Activity flow by thread")
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.scatterplot(data=df,
       x='thread',
       y='module', size='elapsed')
ax.set_title("Thread / Module coupling")
plt.show()

In [ ]:
df.query('"QueueConsumer.run()" != name').groupby(by=['thread']).sum('elapsed').plot(
    y='elapsed', kind="pie", title="elapsed time by thread",
    legend=False, figsize=(6, 6))
plt.show()

20 most called functions

In [ ]:
df['name'].value_counts().head(20)

20 longest single function exection times on the main thread

In [ ]:
df[df.thread == "MainThread"].sort_values(by=['elapsed'], ascending=False).head(20)